# Import Module

In [1]:
import numpy as np
import os
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import tensorflow_hub as hub

Module yang digunakan untuk adalah numpy os, dan tensorflow

# Google Drive Mounting

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Mounting ke google drive pribadi untuk mengambil gambar enoki dan amanita

# Model Inference Function

In [3]:
def import_and_predict(image_data, model):
    target_size = (224, 224)  # Set the target size to match the model's input shape
    image = load_img(image_data, target_size=target_size)  # Load and resize the image
    img_array = img_to_array(image)
    img_array = np.expand_dims(img_array, 0)  # Create a batch

    # Normalize the image
    img_array = img_array / 255.0

    # Make prediction
    predictions = model.predict(img_array)

    # Get the class with the highest probability
    predicted_class = np.where(predictions > 0.2, 1, 0)[0][0]


    class_names = ['edible mushroom sporocarp', 'poisonous mushroom sporocarp']

    result = f"Prediction: {class_names[predicted_class]}"

    return result

Function ini digunakan muntuk import gambar dari google drive dan melakukan prediksi. Hasil dari function ini adalah class prediksi gambar

# Redefine Custom Loss

In [8]:
def binary_focal_loss(gamma=2.0, alpha=0.25):
    def loss(y_true, y_pred):
        epsilon = tf.keras.backend.epsilon()
        y_pred = tf.clip_by_value(y_pred, epsilon, 1. - epsilon)
        cross_entropy = -y_true * tf.math.log(y_pred) - (1 - y_true) * tf.math.log(1 - y_pred)
        weight = alpha * tf.pow(1 - y_pred, gamma) * y_true + (1 - alpha) * tf.pow(y_pred, gamma) * (1 - y_true)
        return tf.reduce_mean(weight * cross_entropy)
    return loss

Function binary_focal_loss perlu di define ulang karena ini adalah custom loss

# Initialize custom loss

In [11]:
custom_loss_instance = binary_focal_loss(gamma=2.0, alpha=0.25)

initialize custom_loss_instance dengan parameter yang digunakan saat training.

# Model Prediction

In [13]:
# Load model for inference
model = load_model(
    'model_E31.keras',
    custom_objects={
        'KerasLayer': hub.KerasLayer,
        'loss': custom_loss_instance
    }
)
# import and predict
import_and_predict('/content/drive/My Drive/mushroom_inference/enoki.jpg',model)

1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step


'Prediction: poisonous mushroom sporocarp'

In [15]:
import_and_predict('/content/drive/My Drive/mushroom_inference/amanita.jpg',model)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 596ms/step


'Prediction: poisonous mushroom sporocarp'

Model memprediksi jamur enoki sebagai jamur beracun yang sebenarnya jamur tersebut aman dimanakan tetapi model dapat memprediksi jamur amanita sebagai jamur yang beracun.